In [148]:
import yfinance as yf
import pandas as pd
from pathlib import Path
import math

In [149]:
result_folder = Path.cwd().parent / 'Results'

symbols = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker'].tolist()
symbols = pd.Series(symbols).unique()
backup = pd.read_csv(result_folder / 'simple_screener_results.csv')['Ticker']
if len(backup) > 0:
    backup.to_csv(result_folder / 'known_tickers.csv', index=False)
owned_tickers = pd.read_csv(result_folder / 'owned_tickers.csv')['Ticker'].tolist()

In [150]:
df = pd.DataFrame(columns=[
    'Ticker', 
    'short_name', 
    'in_portfolio',
    'value_score',  
    'recommendation_score',
    'analyst_mean%', 
    'insider_buy%', 
    'sector', 
    'industry', 
    'P/E (TTM)', 
    'ROA%',
])

for symbol in symbols: 
    ticker = yf.Ticker(symbol)
    try:
        in_portfolio = 'No'
        if symbol in owned_tickers:
            in_portfolio = 'Yes'

        analyst_mean = 0
        if 'mean' in ticker.analyst_price_targets and ticker.info['currentPrice']:
            analyst_mean = ticker.analyst_price_targets['mean'] / ticker.info['currentPrice'] - 1
        
        roa = ticker.info['returnOnAssets']*100
        pe = ticker.info['trailingPE']
        
        value_score = format(2.5 + math.log(roa+5) - math.log(pe+25), '.2f')
        
        insider_buy = 0
        if 'Shares' in ticker.insider_purchases.columns:
            insider_buy = format(ticker.insider_purchases.loc[ticker.insider_purchases.index[4], 'Shares'], '.2f')
            if insider_buy == '<NA>':
                insider_buy = 0

        recommendation_score = format(float(value_score) + analyst_mean * 2.5 + float(insider_buy) * 2.0, '.2f')
        


        df.loc[symbol] = [
            symbol, 
            ticker.info['shortName'], 
            in_portfolio,
            value_score,
            recommendation_score,
            format(analyst_mean * 100, '.2f'), 
            insider_buy,
            ticker.info['sector'], 
            ticker.info['industry'], 
            format(pe, '.2f') if isinstance(pe, (int, float)) else 'N/A', 
            format(roa, '.2f') if isinstance(roa, (int, float)) else 'N/A',
        ]
    except Exception as e:
        print(f"Error with {symbol}: {e}. Removed Ticker from list.")

df.to_csv(result_folder / 'simple_screener_results.csv', index=False)
display(df)

print(f'Got data for {len(df)} stocks')

,Ticker,short_name,in_portfolio,value_score,recommendation_score,analyst_mean%,insider_buy%,sector,industry,P/E (TTM),ROA%
MATAS.CO,MATAS.CO,Matas A/S,No,0.85,1.74,35.54,0,Consumer Cyclical,Specialty Retail,21.05,3.84
TRIFOR.CO,TRIFOR.CO,Trifork Group AG,No,0.75,1.75,39.92,0,Technology,Information Technology Services,12.78,1.59
QQ.L,QQ.L,QINETIQ GROUP PLC ORD 1P,Yes,1.13,1.04,-3.48,0,Industrials,Aerospace & Defense,21.67,6.89
RNMBY,RNMBY,Rheinmetall AG,Yes,0.02,0.02,0.00,0,Industrials,Aerospace & Defense,106.47,5.97
DRS,DRS,"Leonardo DRS, Inc.",No,0.63,1.27,25.67,0.00,Industrials,Aerospace & Defense,37.30,4.64
SAABF,SAABF,Saab AB,Yes,0.36,0.36,0.00,0,Industrials,Aerospace & Defense,49.01,3.70
BCKIY,BCKIY,Babcock International Group Plc,No,0.97,0.97,0.00,0,Industrials,Engineering & Construction,21.31,5.08
BAESY,BAESY,BAE Systems PLC,No,0.81,0.54,-10.89,0,Industrials,Aerospace & Defense,25.76,4.40
IVSO.ST,IVSO.ST,INVISIO AB,No,1.26,0.90,-14.58,0,Technology,Communication Equipment,62.74,20.30
NSKFF,NSKFF,Kongsberg Gruppen ASA,Yes,0.50,0.50,0.00,0,Industrials,Aerospace & Defense,61.45,6.66


Got data for 28 stocks


In [151]:
print(yf.Ticker('RNMBY').info.keys())

dict_keys(['address1', 'city', 'zip', 'country', 'phone', 'fax', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'fullTimeEmployees', 'companyOfficers', 'compensationAsOfEpochDate', 'executiveTeam', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'dividendRate', 'dividendYield', 'exDividendDate', 'payoutRatio', 'fiveYearAvgDividendYield', 'beta', 'trailingPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'trailingAnnualDividendRate', 'trailingAnnualDividendYield', 'currency', 'tradeable', 'enterpriseValue', 'profitMargins', 'floatShares', 'sharesOutstanding', 'heldPercentInsiders', 'heldPer